# Action Recognition @ UCF101  
**Due date: 11:59 pm on Nov. 19, 2019 (Tuesday)**

## Description
---
In this homework, you will be doing action recognition using Recurrent Neural Network (RNN), (Long-Short Term Memory) LSTM in particular. You will be given a dataset called UCF101, which consists of 101 different actions/classes and for each action, there will be 145 samples. We tagged each sample into either training or testing. Each sample is supposed to be a short video, but we sampled 25 frames from each videos to reduce the amount of data. Consequently, a training sample is an image tuple that forms a 3D volume with one dimension encoding *temporal correlation* between frames and a label indicating what action it is.

To tackle this problem, we aim to build a neural network that can not only capture spatial information of each frame but also temporal information between frames. Fortunately, you don't have to do this on your own. RNN — a type of neural network designed to deal with time-series data — is right here for you to use. In particular, you will be using LSTM for this task.

Instead of training an end-to-end neural network from scratch whose computation is prohibitively expensive, we divide this into two steps: feature extraction and modelling. Below are the things you need to implement for this homework:
- **{35 pts} Feature extraction**. Use any of the [pre-trained models](https://pytorch.org/docs/stable/torchvision/models.html) to extract features from each frame. Specifically, we recommend not to use the activations of the last layer as the features tend to be task specific towards the end of the network. 
    **hints**: 
    - A good starting point would be to use a pre-trained VGG16 network, we suggest first fully connected layer `torchvision.models.vgg16` (4096 dim) as features of each video frame. This will result into a 4096x25 matrix for each video. 
    - Normalize your images using `torchvision.transforms` 
    ```
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    prep = transforms.Compose([ transforms.ToTensor(), normalize ])
    prep(img)
    The mean and std. mentioned above is specific to Imagenet data
    
    ```
    More details of image preprocessing in PyTorch can be found at http://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    
- **{35 pts} Modelling**. With the extracted features, build an LSTM network which takes a **dx25** sample as input (where **d** is the dimension of the extracted feature for each frame), and outputs the action label of that sample.
- **{20 pts} Evaluation**. After training your network, you need to evaluate your model with the testing data by computing the prediction accuracy **(5 points)**. The baseline test accuracy for this data is 75%, and **10 points** out of 20 is for achieving test accuracy greater than the baseline. Moreover, you need to compare **(5 points)** the result of your network with that of support vector machine (SVM) (stacking the **dx25** feature matrix to a long vector and train a SVM).
- **{10 pts} Report**. Details regarding the report can be found in the submission section below.

Notice that the size of the raw images is 256x340, whereas your pre-trained model might take **nxn** images as inputs. To solve this problem, instead of resizing the images which unfavorably changes the spatial ratio, we take a better solution: Cropping five **nxn** images, one at the image center and four at the corners and compute the **d**-dim features for each of them, and average these five **d**-dim feature to get a final feature representation for the raw image.
For example, VGG takes 224x224 images as inputs, so we take the five 224x224 croppings of the image, compute 4096-dim VGG features for each of them, and then take the mean of these five 4096-dim vectors to be the representation of the image.

In order to save you computational time, you need to do the classification task only for **the first 25** classes of the whole dataset. The same applies to those who have access to GPUs. **Bonus 10 points for running and reporting on the entire 101 classes.**


## Dataset
Download **dataset** at [UCF101](http://vision.cs.stonybrook.edu/~yangwang/public/UCF101_images.tar)(Image data for each video) and the **annos folder** which has the video labels and the label to class name mapping is included in the assignment folder uploaded. 


UCF101 dataset contains 101 actions and 13,320 videos in total.  

+ `annos/actions.txt`  
  + lists all the actions (`ApplyEyeMakeup`, .., `YoYo`)   
  
+ `annots/videos_labels_subsets.txt`  
  + lists all the videos (`v_000001`, .., `v_013320`)  
  + labels (`1`, .., `101`)  
  + subsets (`1` for train, `2` for test)  

+ `images/`  
  + each folder represents a video
  + the video/folder name to class mapping can be found using `annots/videos_labels_subsets.txt`, for e.g. `v_000001` belongs to class 1 i.e. `ApplyEyeMakeup`
  + each video folder contains 25 frames  



## Some Tutorials
- Good materials for understanding RNN and LSTM
    - http://blog.echen.me
    - http://karpathy.github.io/2015/05/21/rnn-effectiveness/
    - http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Implementing RNN and LSTM with PyTorch
    - [LSTM with PyTorch](http://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py)
    - [RNN with PyTorch](http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

---
---
## **Problem 1.** Feature extraction

In [0]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pwd

/home/gautham/Desktop/cv_hw5/UCF101_images


In [0]:
import os
os.chdir('/content/gdrive/My Drive/Ramajayam_Gautham_112965275_hw5')

In [0]:
#!conda install pytorch=1.0.1 -c pytorch
import torch 
print(torch.__version__)

1.3.1


In [0]:
#import os
import tarfile
import time
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

import numpy as np
import scipy as sp
import cv2
import glob
import pdb


#tar = tarfile.open('./data/UCF101_images.tar',"r:")
#tar.extractall()
#tar.close()

In [0]:
df = pd.read_csv('./annos/videos_labels_subsets.txt', sep="\t", header=None, names=["folder", "class_id", "data_usage"])
#df.head()
classes = pd.read_csv('./annos/actions.txt', sep="\t", header = None)

df_small = df[df.class_id <= 25]

df_small_train = df_small[df_small.data_usage == 1]
df_small_test = df_small[df_small.data_usage == 2]

train25_folders = df_small_train.folder.tolist()
train25_labels = df_small_train.class_id.tolist()

test25_folders = df_small_test.folder.tolist()
test25_labels = df_small_test.class_id.tolist()
#classes

df_train = df[df.data_usage == 1]
df_test = df[df.data_usage == 2]

train_folders = df_train.folder.tolist()
train_labels = df_train.class_id.tolist()

test_folders = df_test.folder.tolist()
test_labels = df_test.class_id.tolist()

print(train25_folders)

['v_000045', 'v_000046', 'v_000047', 'v_000048', 'v_000049', 'v_000050', 'v_000051', 'v_000052', 'v_000053', 'v_000054', 'v_000055', 'v_000056', 'v_000057', 'v_000058', 'v_000059', 'v_000060', 'v_000061', 'v_000062', 'v_000063', 'v_000064', 'v_000065', 'v_000066', 'v_000067', 'v_000068', 'v_000069', 'v_000070', 'v_000071', 'v_000072', 'v_000073', 'v_000074', 'v_000075', 'v_000076', 'v_000077', 'v_000078', 'v_000079', 'v_000080', 'v_000081', 'v_000082', 'v_000083', 'v_000084', 'v_000085', 'v_000086', 'v_000087', 'v_000088', 'v_000089', 'v_000090', 'v_000091', 'v_000092', 'v_000093', 'v_000094', 'v_000095', 'v_000096', 'v_000097', 'v_000098', 'v_000099', 'v_000100', 'v_000101', 'v_000102', 'v_000103', 'v_000104', 'v_000105', 'v_000106', 'v_000107', 'v_000108', 'v_000109', 'v_000110', 'v_000111', 'v_000112', 'v_000113', 'v_000114', 'v_000115', 'v_000116', 'v_000117', 'v_000118', 'v_000119', 'v_000120', 'v_000121', 'v_000122', 'v_000123', 'v_000124', 'v_000125', 'v_000126', 'v_000127', 'v_

In [0]:
from torch.utils.data import Dataset,DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

import gc

class UCF101Dataset(Dataset):
    
    def __init__(self, data_list, labels_list, root_dir, vgg16):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #self.dataframe = dataframe
        self.data_list = data_list
        self.labels_list = labels_list
        self.root_dir = root_dir
        self.normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        self.prep = torchvision.transforms.Compose([ torchvision.transforms.ToTensor(), self.normalize ])
        #self.transform = transform
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        #self.vgg16 = vgg16
        print(self.device)
        self.vgg16 = vgg16.to(self.device)
        
        del self.vgg16.classifier[1:]
        print(self.vgg16.classifier)
        self.length = len(self.data_list)*25
        #print("init", self.length)

    def __len__(self):
        #print("len")
        return (self.length)

    def __getitem__(self, idx):
        #if torch.is_tensor(idx):
        #    idx = idx.tolist()
        #print("folder")
        folder_name = os.path.join(self.root_dir,
                                self.data_list[idx//25])
        
        files = os.listdir(folder_name)
        #print(files)
        #for i in range(len(files)):
          
        file_name = os.path.join(folder_name, files[idx%25])
          
        #img_name = img_name + '/*.jpg' 
        #image = cv2.imread(file_name)
        label = self.labels_list[idx//25]

        #if idx % 25 == 0:
          #print(folder_name, label)
        #gc.collect()
        torch.cuda.empty_cache()

        #print(file_name)
        img = cv2.imread(file_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.prep(img)
        #print(img.size()[0])
        img_c = img[ :, (img.shape[1]-224)//2:(img.shape[1]+224)//2, (img.shape[2]-224)//2:(img.shape[2]+224)//2]
        img_tl = img[:, :224, :224] 
        img_tr = img[:, :224, (img.shape[2]-224):]
        img_bl = img[:, (img.shape[1]-224):, :224] 
        img_br = img[:, (img.shape[1]-224):, (img.shape[2]-224):] 
        #print(len(img_tr[2]), len(img_bl[1]), len(img_br[0]))
        #print(img_c.size(), img_tl.size(), img_tr.size(), img_bl.size(), img_br.size())
        with torch.no_grad():
          inputs = torch.stack([img_c, img_tl, img_tr, img_bl, img_br])
          #print(len(inputs))
          inputs = inputs.to(self.device)
          #img_feat = self.vgg16(inputs)
          vgg16_local = self.vgg16
          img_feat1 = vgg16_local(inputs)
          img_feat1 = img_feat1.cpu()
          #print(inputs.is_cuda, img_feat1.is_cuda, img_c.is_cuda)#, img_feat1.is_cuda)
          del inputs, vgg16_local
          torch.cuda.empty_cache()
          #gc.collect()
          #img_feat = self.vgg16(img_c)
          #print(len(img_feat), len(img_feat[0]),len(img_feat[1]),len(img_feat[2]))
          img_feat1 = img_feat1.mean(dim=0)
          #print(len(img_feat1))
          #torch.cuda.empty_cache()
          

        return (img_feat1, label)
        #landmarks = np.array([landmarks])
        #landmarks = landmarks.astype('float').reshape(-1, 2)
        #sample = {'image': image, 'landmarks': landmarks}

        

    

cuda


In [0]:
from torch.utils.data import Dataset,DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

import gc

class UCF101Dataset(Dataset):
    
    def __init__(self, data_list, labels_list, root_dir, vgg16):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #self.dataframe = dataframe
        self.data_list = data_list
        self.labels_list = labels_list
        self.root_dir = root_dir
        self.normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        self.prep = torchvision.transforms.Compose([ torchvision.transforms.ToTensor(), self.normalize ])
        #self.transform = transform
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        #self.vgg16 = vgg16
        print(self.device)
        self.vgg16 = vgg16.to(self.device)
        
        del self.vgg16.classifier[1:]
        print(self.vgg16.classifier)
        self.length = len(self.data_list)
        #print("init", self.length)

    def __len__(self):
        #print("len")
        return (self.length)

    def __getitem__(self, idx):
        #if torch.is_tensor(idx):
        #    idx = idx.tolist()
        #print("folder")
        folder_name = os.path.join(self.root_dir,
                                self.data_list[idx])
        label = self.labels_list[idx]
        
        files = os.listdir(folder_name)
        #print(files)
        #for i in range(len(files)):
        i = 0
        feats25 = torch.empty(0)
        while i < 25:
          
          file_name = os.path.join(folder_name, files[i])
            
          #img_name = img_name + '/*.jpg' 
          #image = cv2.imread(file_name)
          

          #if idx % 25 == 0:
            #print(folder_name, label)
          gc.collect()
          torch.cuda.empty_cache()

          #print(file_name)
          img = cv2.imread(file_name)
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          img = self.prep(img)
          #print(img.size()[0])
          img_c = img[ :, (img.shape[1]-224)//2:(img.shape[1]+224)//2, (img.shape[2]-224)//2:(img.shape[2]+224)//2]
          img_tl = img[:, :224, :224] 
          img_tr = img[:, :224, (img.shape[2]-224):]
          img_bl = img[:, (img.shape[1]-224):, :224] 
          img_br = img[:, (img.shape[1]-224):, (img.shape[2]-224):] 
          #print(len(img_tr[2]), len(img_bl[1]), len(img_br[0]))
          #print(img_c.size(), img_tl.size(), img_tr.size(), img_bl.size(), img_br.size())
          with torch.no_grad():
            inputs = torch.stack([img_c, img_tl, img_tr, img_bl, img_br])
            #print(len(inputs))
            inputs = inputs.to(self.device)
            #img_feat = self.vgg16(inputs)
            vgg16_local = self.vgg16
            img_feat1 = vgg16_local(inputs)
            img_feat1 = img_feat1.cpu()
            #print(inputs.is_cuda, img_feat1.is_cuda, img_c.is_cuda)#, img_feat1.is_cuda)
            del inputs, vgg16_local
            torch.cuda.empty_cache()
            #gc.collect()
            #img_feat = self.vgg16(img_c)
            #print(len(img_feat), len(img_feat[0]),len(img_feat[1]),len(img_feat[2]))
            img_feat1 = img_feat1.mean(dim=0)
            if i == 0:
              feats25 = img_feat1
            else:  
              feats25 = torch.cat((feats25, img_feat1), 0)
            del img_feat1  
            torch.cuda.empty_cache()
            gc.collect()
            #print(len(img_feat1))
            #torch.cuda.empty_cache()

            i += 1
          
        #print(len(feats25), label)
        return (feats25, label)
        #landmarks = np.array([landmarks])
        #landmarks = landmarks.astype('float').reshape(-1, 2)
        #sample = {'image': image, 'landmarks': landmarks}

        

    

cuda


In [0]:
print(len(train25_folders), len(train25_labels))
train25_folders1 = train25_folders[:500]
train25_folders2 = train25_folders[500:1000]
train25_folders3 = train25_folders[1000:1500]
train25_folders4 = train25_folders[1500:2000]
train25_folders5 = train25_folders[2000:]
print(len(train25_folders1), len(train25_folders2), len(train25_folders3), len(train25_folders4), len(train25_folders5))

train25_labels1 = train25_labels[:500]
train25_labels2 = train25_labels[500:1000]
train25_labels3 = train25_labels[1000:1500]
train25_labels4 = train25_labels[1500:2000]
train25_labels5 = train25_labels[2000:]
print(len(train25_labels1), len(train25_labels2), len(train25_labels3), len(train25_labels4), len(train25_labels5))

print(train25_folders1) 
print(train25_labels1)
print(train25_folders2)
print(train25_labels2)


2409 2409
500 500 500 500 409
500 500 500 500 409
['v_000045', 'v_000046', 'v_000047', 'v_000048', 'v_000049', 'v_000050', 'v_000051', 'v_000052', 'v_000053', 'v_000054', 'v_000055', 'v_000056', 'v_000057', 'v_000058', 'v_000059', 'v_000060', 'v_000061', 'v_000062', 'v_000063', 'v_000064', 'v_000065', 'v_000066', 'v_000067', 'v_000068', 'v_000069', 'v_000070', 'v_000071', 'v_000072', 'v_000073', 'v_000074', 'v_000075', 'v_000076', 'v_000077', 'v_000078', 'v_000079', 'v_000080', 'v_000081', 'v_000082', 'v_000083', 'v_000084', 'v_000085', 'v_000086', 'v_000087', 'v_000088', 'v_000089', 'v_000090', 'v_000091', 'v_000092', 'v_000093', 'v_000094', 'v_000095', 'v_000096', 'v_000097', 'v_000098', 'v_000099', 'v_000100', 'v_000101', 'v_000102', 'v_000103', 'v_000104', 'v_000105', 'v_000106', 'v_000107', 'v_000108', 'v_000109', 'v_000110', 'v_000111', 'v_000112', 'v_000113', 'v_000114', 'v_000115', 'v_000116', 'v_000117', 'v_000118', 'v_000119', 'v_000120', 'v_000121', 'v_000122', 'v_000123', '

In [0]:
test25_folders1 = test25_folders[:500]
test25_folders2 = test25_folders[500:]

test25_labels1 = test25_labels[:500]
test25_labels2 = test25_labels[500:]

print(len(test25_labels2))

451


In [0]:
"""
import time
import gc
gc.collect()
if __name__ == '__main__':
  torch.cuda.empty_cache()
  #kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}
  test25_object1 = UCF101Dataset(test25_folders1, test25_labels1, './images/', torchvision.models.vgg16(True))
  test25_dataloader1 = DataLoader(test25_object1, batch_size=25,
                        shuffle=False, num_workers = 0, pin_memory = True)
  i = 0

  start = time.time()
  for a in test25_dataloader1:
        print(a)
        i += 1      
        print("Folders done", i)
        print(time.time() - start)
"""        

In [0]:
"""

import time
import gc
gc.collect()
if __name__ == '__main__':
  torch.cuda.empty_cache()
  #kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}
  test25_object2 = UCF101Dataset(test25_folders2, test25_labels2, './images/', torchvision.models.vgg16(True))
  test25_dataloader2 = DataLoader(test25_object2, batch_size=25,
                        shuffle=False, num_workers = 0, pin_memory = True)
  i = 0

  start = time.time()
  for a in test25_dataloader2:
        print(a)
        i += 1      
        print("Folders done", i)
        print(time.time() - start)

gc.collect()
torch.cuda.empty_cache()

import pickle              # import module first

f = open('test25_dataloader2.pkl', 'wb')   # Pickle file is newly created where foo1.py is
pickle.dump(test25_dataloader2, f, -1)          # dump data to f
f.close()
"""

In [0]:
"""
import pickle              # import module first

f = open('test25_dataloader1.pkl', 'wb')   # Pickle file is newly created where foo1.py is
pickle.dump(test25_dataloader1, f, -1)          # dump data to f
f.close()
"""

In [0]:
"""
#del vgg16.classifier[1:]
import time

if __name__ == '__main__':
  torch.cuda.empty_cache()
  #kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}
  train25_object1 = UCF101Dataset(train25_folders1, train25_labels1, './images/', torchvision.models.vgg16(True))
  train25_dataloader1 = DataLoader(train25_object1, batch_size=1250,
                        shuffle=False, num_workers = 0, pin_memory = True)
  i = 0

  start = time.time()
  for a in train25_dataloader1:
        print(a)
        i += 50      
        print("Folders done", i)
        print(time.time() - start)
"""        
    

In [0]:
"""
if __name__ == '__main__':
  torch.cuda.empty_cache()
  #kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}
  train25_object2 = UCF101Dataset(train25_folders2, train25_labels2, './images/', torchvision.models.vgg16(True))
  train25_dataloader2 = DataLoader(train25_object2, batch_size=1250,
                        shuffle=False, num_workers = 0, pin_memory = True)
  i = 500

  start = time.time()
  for _, a in enumerate(train25_dataloader2):
        print(a)
        i += 50      
        print("Folders done", i)
        print(time.time() - start)
"""        

In [0]:
torch.cuda.empty_cache()

In [0]:
"""
if __name__ == '__main__':
  torch.cuda.empty_cache()
  #kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}
  train25_object3 = UCF101Dataset(train25_folders3, train25_labels3, './images/', torchvision.models.vgg16(True))
  train25_dataloader3 = DataLoader(train25_object3, batch_size=1250,
                        shuffle=False, num_workers = 0, pin_memory = True)
  i = 1000

  start = time.time()
  for _, a in enumerate(train25_dataloader3):
        print(a)
        i += 50      
        print("Folders done", i)
        print(time.time() - start)
"""
             

In [0]:
"""
import pickle              # import module first

f = open('train25_dataloader3.pkl', 'wb')   # Pickle file is newly created where foo1.py is
pickle.dump(train25_dataloader3, f, -1)          # dump data to f
f.close()
"""

In [0]:
"""
if __name__ == '__main__':

  torch.cuda.empty_cache()
  #kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}
  train25_object4 = UCF101Dataset(train25_folders4, train25_labels4, './images/', torchvision.models.vgg16(True))
  train25_dataloader4 = DataLoader(train25_object4, batch_size=1250,
                        shuffle=False, num_workers = 0, pin_memory = True)
  i = 1500

  start = time.time()
  for _, b in enumerate(train25_dataloader4):
        print(b)
        i += 50      
        print("Folders done", i)
        print(time.time() - start)

 """ 

In [0]:
"""
#del vgg16.classifier[1:]
#import time

if __name__ == '__main__':
  torch.cuda.empty_cache()
  #kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}
  train25_object5 = UCF101Dataset(train25_folders5, train25_labels5, './images/', torchvision.models.vgg16(True))
  train25_dataloader5_1 = DataLoader(train25_object5, batch_size=1,
                        shuffle=False, num_workers = 0, pin_memory = True)
  i = 0

  start = time.time()
  for a in train25_dataloader5_1:
        gc.collect()
        #print(a)
        i += 1      
        print("Folders done", i, time.time() - start)
        #gc.collect()
        torch.cuda.empty_cache()
        #print(time.time() - start)

gc.collect()
torch.cuda.empty_cache()

import pickle              # import module first

f = open('train25_dataloader5_1.pkl', 'wb')   # Pickle file is newly created where foo1.py is
pickle.dump(train25_dataloader5_1, f, -1)          # dump data to f
f.close()
"""    

cuda
Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
)
Folders done 1 2.7607970237731934
Folders done 2 5.534983158111572
Folders done 3 14.899595022201538
Folders done 4 27.64732551574707
Folders done 5 38.276832580566406
Folders done 6 49.93973398208618
Folders done 7 61.76861000061035
Folders done 8 73.76342463493347
Folders done 9 93.53428721427917
Folders done 10 105.90463781356812
Folders done 11 118.29985117912292
Folders done 12 128.79759335517883
Folders done 13 140.00290870666504
Folders done 14 150.9299876689911
Folders done 15 161.69674062728882
Folders done 16 172.02588510513306
Folders done 17 182.19706535339355
Folders done 18 194.6644332408905
Folders done 19 205.78002738952637
Folders done 20 216.78631496429443
Folders done 21 228.39944052696228
Folders done 22 239.98511481285095
Folders done 23 251.45474576950073
Folders done 24 263.0483546257019
Folders done 25 273.57896065711975
Folders done 26 286.6323256492615
Folders done 27 298.5509028

OSError: ignored

In [0]:
"""
import pickle              # import module first

f = open('train25_dataloader5_10folders.pkl', 'wb')   # Pickle file is newly created where foo1.py is
pickle.dump(train25_dataloader5, f, -1)          # dump data to f
f.close()
"""

In [0]:
"""
import pickle

f = open('train25_dataloader1.pkl', 'rb')   # 'rb' for reading binary file
train25_dataloader1_pkl = pickle.load(f)     
f.close() 
"""

In [0]:

"""
f = open('train25_dataloader2.pkl', 'rb')   # 'rb' for reading binary file
train25_dataloader2_pkl = pickle.load(f)     
f.close() 

f = open('train25_dataloader3.pkl', 'rb')   # 'rb' for reading binary file
train25_dataloader3_pkl = pickle.load(f)     
f.close() 

f = open('train25_dataloader4.pkl', 'rb')   # 'rb' for reading binary file
train25_dataloader4_pkl = pickle.load(f)     
f.close() 

f = open('train25_dataloader5_10folders.pkl', 'rb')   # 'rb' for reading binary file
train25_dataloader5_10_pkl = pickle.load(f)     
f.close()                       

print(len(train25_dataloader3_pkl))
"""

10


In [0]:
import gc
gc.collect()
#torch.cuda.empty_cache()

0

In [0]:
"""
train25_dataloader5_50_trial = DataLoader(train25_dataloader5_10_pkl, batch_size=5,
                        shuffle=False, num_workers = 0, pin_memory = True)

f = open('train25_dataloader5_50_trial.pkl', 'wb')   # Pickle file is newly created where foo1.py is
pickle.dump(train25_dataloader5_50_trial, f, -1)          # dump data to f
f.close()

print(len(train25_dataloader5_50_trial))
"""

9


In [0]:
"""
f = open('train25_dataloader5_50_trial.pkl', 'rb')   # 'rb' for reading binary file
train25_dataloader5_50_trial_pkl = pickle.load(f)     
f.close() 
"""  

In [0]:
#print(train25_object5[1000])
features25_list = []
labels_list_tot = []

In [0]:
"""
j = 0
for feats, labels in train25_dataloader1_pkl:
  #print(feats.size())
  i = 0
  #j =
  while i < (feats.size()[0]//25):
    feat_reshaped = torch.reshape(feats[25*i:25*(i+1)], (-1, 102400))
    labels_reshaped = torch.reshape(labels[25*i:25*(i+1)], (-1, 25))
    features25_list.append(feat_reshaped[0])
    labels_list_tot.append(labels_reshaped[0][0]) 
    #print(feat_reshaped.size(), labels_reshaped.size())
    #print(feat_reshaped, labels_reshaped[0][0])
    #print(len(features25_list), len(features25_list[0]))
    i += 1
    gc.collect()

  j += 50
  print(j, len(features25_list))
  """



50 50
100 100
150 150
200 200
250 250
300 300
350 350
400 400
450 450
500 500


In [0]:
#import gc
gc.collect()
#torch.cuda.empty_cache()

0

In [0]:
"""
import pickle

f = open('features25_list_1.pkl', 'wb')   # Pickle file is newly created where foo1.py is
pickle.dump(features25_list, f, -1)          # dump data to f
f.close()
"""


In [0]:
gc.collect()
#torch.cuda.empty_cache()

0

In [0]:
"""
f = open('labels_list_tot_1.pkl', 'wb')   # Pickle file is newly created where foo1.py is
pickle.dump(labels_list_tot, f, -1)          # dump data to f
f.close()
"""

In [0]:
#features25_list_1_pkl = []

In [0]:
"""
import pickle

f = open('features25_list_4.pkl', 'rb')   # 'rb' for reading binary file
features25_list_3_pkl = pickle.load(f)     
f.close()
""" 

In [0]:
"""
f = open('labels_list_tot_4.pkl', 'rb')   # 'rb' for reading binary file
labels_list_tot_3_pkl = pickle.load(f)     
f.close() 
"""

In [0]:
gc.collect()

0

In [0]:
"""
for feats, labels in train25_dataloader2_pkl:
  #print(feats.size())
  i = 0
  while i < (feats.size()[0]//25):
    feat_reshaped = torch.reshape(feats[25*i:25*(i+1)], (-1, 102400))
    labels_reshaped = torch.reshape(labels[25*i:25*(i+1)], (-1, 25))
    features25_list.append(feat_reshaped[0])
    labels_list_tot.append(labels_reshaped[0][0]) 
    #print(feat_reshaped.size(), labels_reshaped.size())
    #print(feat_reshaped, labels_reshaped[0][0])
    #print(len(features25_list), len(features25_list[0]))
    i += 1
  print(j+50, len(features25_list))
  f = open('features25_list.pkl', 'wb')   # Pickle file is newly created where foo1.py is
  pickle.dump(features25_list, f, -1)          # dump data to f
  f.close()

  f = open('labels_list_tot.pkl', 'wb')   # Pickle file is newly created where foo1.py is
  pickle.dump(labels_list_tot, f, -1)          # dump data to f
  f.close()
"""


In [0]:
"""
for feats, labels in train25_dataloader3_pkl:
  #print(feats.size())
  i = 0
  while i < (feats.size()[0]//25):
    feat_reshaped = torch.reshape(feats[25*i:25*(i+1)], (-1, 102400))
    labels_reshaped = torch.reshape(labels[25*i:25*(i+1)], (-1, 25))
    features25_list.append(feat_reshaped[0])
    labels_list_tot.append(labels_reshaped[0][0]) 
    #print(feat_reshaped.size(), labels_reshaped.size())
    #print(feat_reshaped, labels_reshaped[0][0])
    #print(len(features25_list), len(features25_list[0]))
    i += 1
  print(j+50, len(features25_list))  
  f = open('features25_list.pkl', 'wb')   # Pickle file is newly created where foo1.py is
  pickle.dump(features25_list, f, -1)          # dump data to f
  f.close()

  f = open('labels_list_tot.pkl', 'wb')   # Pickle file is newly created where foo1.py is
  pickle.dump(labels_list_tot, f, -1)          # dump data to f
  f.close()  
"""


In [0]:
"""
for feats, labels in train25_dataloader4_pkl:
  #print(feats.size())
  i = 0
  while i < (feats.size()[0]//25):
    feat_reshaped = torch.reshape(feats[25*i:25*(i+1)], (-1, 102400))
    labels_reshaped = torch.reshape(labels[25*i:25*(i+1)], (-1, 25))
    features25_list.append(feat_reshaped[0])
    labels_list_tot.append(labels_reshaped[0][0]) 
    #print(feat_reshaped.size(), labels_reshaped.size())
    #print(feat_reshaped, labels_reshaped[0][0])
    #print(len(features25_list), len(features25_list[0]))
    i += 1
  print(j+50, len(features25_list))  
  f = open('features25_list.pkl', 'wb')   # Pickle file is newly created where foo1.py is
  pickle.dump(features25_list, f, -1)          # dump data to f
  f.close()

  f = open('labels_list_tot.pkl', 'wb')   # Pickle file is newly created where foo1.py is
  pickle.dump(labels_list_tot, f, -1)          # dump data to f
  f.close()     
"""


In [0]:
"""
for feats, labels in train25_dataloader5_10_pkl:
  #print(feats.size())
  i = 0
  while i < (feats.size()[0]//25):
    feat_reshaped = torch.reshape(feats[25*i:25*(i+1)], (-1, 102400))
    labels_reshaped = torch.reshape(labels[25*i:25*(i+1)], (-1, 25))
    features25_list.append(feat_reshaped[0])
    labels_list_tot.append(labels_reshaped[0][0]) 
    #print(feat_reshaped.size(), labels_reshaped.size())
    #print(feat_reshaped, labels_reshaped[0][0])
    #print(len(features25_list), len(features25_list[0]))
    i += 1
  print(j+10, len(features25_list))  
  f = open('features25_list.pkl', 'wb')   # Pickle file is newly created where foo1.py is
  pickle.dump(features25_list, f, -1)          # dump data to f
  f.close()

  f = open('labels_list_tot.pkl', 'wb')   # Pickle file is newly created where foo1.py is
  pickle.dump(labels_list_tot, f, -1)          # dump data to f
  f.close()

print(len(features25_list))
"""

In [0]:
"""
f = open('features25_list.pkl', 'wb')   # Pickle file is newly created where foo1.py is
pickle.dump(features25_list, f, -1)          # dump data to f
f.close()

f = open('labels_list_tot.pkl', 'wb')   # Pickle file is newly created where foo1.py is
pickle.dump(labels_list_tot, f, -1)          # dump data to f
f.close()
"""

In [0]:
gc.collect()

0

In [0]:
# \*write your codes for feature extraction (You can use multiple cells, this is just a place holder)


***
***
## **Problem 2.** Modelling

* ##### **Print the size of your training and test data**

In [0]:
class LSTM(nn.Module):
    
    def __init__(self, hidden_dim, batch_size, label_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size= batch_size
        
        self.lstm = nn.LSTM(102400, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        return (torch.zeros(1, self.batch_size, self.hidden_dim),
                torch.zeros(1, self.batch_size, self.hidden_dim))
    
    def forward(self, input_features):
        inputs = input_features.view(self.batch_size,-1)
        lstm_out, self.hidden = self.lstm(inputs.view(1,self.batch_size,-1), self.hidden)
        outputs = self.hidden2label(lstm_out.view(self.batch_size,-1))
        output_labels = F.log_softmax(outputs, dim=-1)
        #print(output_labels.shape)
        
        return output_labels
        

In [0]:
import pickle

f = open('features25_list_5.pkl', 'rb')   # 'rb' for reading binary file
features25_list_5_pkl = pickle.load(f)     
f.close() 

print(len(features25_list_5_pkl))

409


In [0]:
f = open('labels_list_tot_5.pkl', 'rb')   # 'rb' for reading binary file
labels_list_tot_5_pkl = pickle.load(f)     
f.close() 

In [0]:
# Don't hardcode the shape of train and test data
print('Shape of training data is :', len(train25_labels),"x", len(features25_list_5_pkl[0]))
print('Shape of test/validation data is :', len(test25_labels), "x", len(features25_list_5_pkl[0]))

#I had divided the complete dataset into 5 parts to parallely extract features initially 

Shape of training data is : 2409 x 102400
Shape of test/validation data is : 951 x 102400


In [0]:
from sklearn.utils import shuffle

#features25_list_2_pkl_shuffled, labels_list_tot_2_pkl_shuffled  = shuffle(features25_list_2_pkl, labels_list_tot_2_pkl)


In [0]:
gc.collect()

0

In [0]:
#Training each part of data separately
#Completele training is also shown below
#This is for network with 512 hidden dimensions

Batch_size = 50
Hidden_dim = 512
Num_classes = 25

train_data = features25_list_5_pkl
train_labels = labels_list_tot_5_pkl

state_dict = torch.load('feat512_1234.pth')

model_1 = LSTM(Hidden_dim,Batch_size,Num_classes)
model_1.load_state_dict(state_dict)
print(model_1)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model_1.parameters(), lr=0.1)

#model_1 = model_1.to(device) 


correct = 0
train_accuracy = 0
for epoch in range(10):
    loss_count = 0.0
    correct = 0
    total = 0
    train_accuracy = 0
    start = time.time()
    #for feats, labels in train25_dataloader2_1_pkl:
    for i in range(len(train_data)//Batch_size):
      #print(feats.size())
      #i = 0
      #while i < (feats.size()[0]//25):
      #  feat_reshaped = torch.reshape(feats[25*i:25*(i+1)], (-1, 102400))
      #  labels_reshaped = torch.reshape(labels[25*i:25*(i+1)], (-1, 25))
        #print(feat_reshaped.size(), labels_reshaped.size())
        #print(feat_reshaped, labels_reshaped[0])
        #feats = input_data[i]
        #labels = label[i]
      inputs  = train_data[i: i+Batch_size]
      inputs = torch.stack(inputs)
      #inputs = inputs.to(device)
      labels = train_labels[i : i+Batch_size]
      labels = torch.stack(labels)
      #print(labels.shape)
      #labels = labels.to(device)
      model_1.zero_grad()
      model_1.hidden= model_1.init_hidden()
      outputs = model_1(inputs)
      #print(outputs.shape, labels.shape)
      loss = loss_function(outputs, labels)
      loss.backward()
      optimizer.step()
      loss_count += loss.item()
      _, predicted = torch.max(outputs.data, 1)
      
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      del outputs, inputs, labels
      #torch.cuda.empty_cache()
      train_accuracy = (correct/total)*100
      
      #i += 1 

    print('Epoch : %d Loss: %.3f' %(epoch+1, loss_count/len(features25_list_5_pkl)))
    print('Training Accuracy', train_accuracy , time.time() - start)

LSTM(
  (lstm): LSTM(102400, 512)
  (hidden2label): Linear(in_features=512, out_features=25, bias=True)
)
Epoch : 1 Loss: 0.026
Training Accuracy 71.75 10.44509768486023
Epoch : 2 Loss: 0.002
Training Accuracy 100.0 10.295786619186401
Epoch : 3 Loss: 0.001
Training Accuracy 100.0 10.450769424438477
Epoch : 4 Loss: 0.000
Training Accuracy 100.0 10.509390354156494
Epoch : 5 Loss: 0.000
Training Accuracy 100.0 10.59251356124878
Epoch : 6 Loss: 0.000
Training Accuracy 100.0 10.676548957824707
Epoch : 7 Loss: 0.000
Training Accuracy 100.0 10.7195143699646
Epoch : 8 Loss: 0.000
Training Accuracy 100.0 10.695135593414307
Epoch : 9 Loss: 0.000
Training Accuracy 100.0 10.703275918960571
Epoch : 10 Loss: 0.000
Training Accuracy 100.0 10.739113807678223


In [0]:
torch.save(model_1.state_dict(), 'feat512_12345.pth')

---
---
## **Problem 3.** Evaluation

In [0]:
import pickle

f = open('features25_list_test_1.pkl', 'rb')   # 'rb' for reading binary file
features25_list_test_1_pkl = pickle.load(f)     
f.close() 



In [0]:
print(len(features25_list_test_1_pkl))

500


In [0]:
f = open('features25_list_test_2.pkl', 'rb')   # 'rb' for reading binary file
features25_list_test_2_pkl = pickle.load(f)     
f.close() 

In [0]:
test_data = features25_list_test_1_pkl + features25_list_test_2_pkl


In [0]:
print(len(features25_list_test_2_pkl))

In [0]:
gc.collect()

0

In [0]:
f = open('labels_list_tot_test_1.pkl', 'rb')   # 'rb' for reading binary file
labels_list_tot_test_1_pkl = pickle.load(f)     
f.close() 

In [0]:
print(len(labels_list_tot_test_1_pkl))

In [0]:
f = open('labels_list_tot_test_2.pkl', 'rb')   # 'rb' for reading binary file
labels_list_tot_test_2_pkl = pickle.load(f)     
f.close() 

In [0]:
test_labels = labels_list_tot_test_1_pkl + labels_list_tot_test_2_pkl


In [0]:
#del features25_list_test_1_pkl, features25_list_test_2_pkl, test_data
#del labels_list_tot_test_1_pkl, labels_list_tot_test_2_pkl, test_labels

In [0]:
from sklearn.utils import shuffle

#features25_list_test_1_pkl_shuffled, labels_list_tot_test_1_pkl_shuffled  = shuffle(features25_list_test_1_pkl, labels_list_tot_test_1_pkl)
test_data = features25_list_test_1_pkl 
test_labels = labels_list_tot_test_1_pkl

In [0]:
import time
test_accuracy = 0

Batch_size = 50
Hidden_dim = 256
Num_classes = 25

correct2 = 0
total2 = 0

state_dict = torch.load('feat5_4_3_s_2_s_1_s.pth')

model = LSTM(Hidden_dim,Batch_size,Num_classes)
model.load_state_dict(state_dict)
#print(model_1)

for epoch in range(1):
    loss_count = 0.0
    correct = 0
    total = 0
    accuracy = 0
    start = time.time()
    #for feats, labels in train25_dataloader2_1_pkl:
    for i in range(len(test_data)//Batch_size):
      
      inputs  = test_data[i: i+Batch_size]
      inputs = torch.stack(inputs)
      #inputs = inputs.to(device)
      labels = test_labels[i : i+Batch_size]
      labels = torch.stack(labels)
      #print(labels.shape)
      #labels = labels.to(device)
      #model_1.zero_grad()
      #model_1.hidden= model_1.init_hidden()
      outputs = model(inputs)
      #print(outputs.shape, labels.shape)
      #loss = loss_function(outputs, labels)
      #loss.backward()
      #optimizer.step()
      #loss_count += loss.item()
      _, predicted = torch.max(outputs.data, 1)
      
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      del outputs, inputs, labels
      torch.cuda.empty_cache()
      test_accuracy = (correct/total)*100
      #print(correct, total)
      
      #i += 1 

    #print('Epoch : %d Loss: %.3f' %(epoch+1, loss_count/len(features25_list_test_1_pkl)))
    print('Testing Accuracy', test_accuracy , time.time() - start)

Testing Accuracy 79.0 1.462536334991455


In [0]:
print(len(features25_list_test_1_pkl))

* ##### **Print the train and test accuracy of your model** 

In [0]:
# Don't hardcode the train and test accuracy
print('Training accuracy is %2.3f :' %(100.00), train_accuracy )
print('Test accuracy is %2.3f :' %(100.00), test_accuracy )

Training accuracy is 100.000 : 100.0
Test accuracy is 100.000 : 79.0


* ##### **Print the train and test and test accuracy of SVM** 

In [0]:
#test
test_data = features25_list_test_1_pkl + features25_list_test_2_pkl
test_labels = labels_list_tot_test_1_pkl + labels_list_tot_test_2_pkl

test_data_numpy = []
test_labels_numpy = []

for i in range(len(test_data)):
  test_data_numpy.append(test_data[i].numpy())
  test_labels_numpy.append(test_labels[i].numpy())

In [0]:
f = open('features25_list_1.pkl', 'rb')   # 'rb' for reading binary file
features25_list_1_pkl = pickle.load(f)     
f.close() 

In [0]:
f = open('features25_list_2.pkl', 'rb')   # 'rb' for reading binary file
features25_list_2_pkl = pickle.load(f)     
f.close() 

In [0]:
f = open('features25_list_3_1.pkl', 'rb')   # 'rb' for reading binary file
features25_list_3_1_pkl = pickle.load(f)     
f.close() 

In [0]:
def accuracy_test1(test_labels, pred_labels):
  return 2*(test_labels == pred_labels).mean()


In [0]:
f = open('features25_list_4_1.pkl', 'rb')   # 'rb' for reading binary file
features25_list_4_1_pkl = pickle.load(f)     
f.close()

In [0]:
f = open('features25_list_5.pkl', 'rb')   # 'rb' for reading binary file
features25_list_5_pkl = pickle.load(f)     
f.close()

In [0]:
f = open('labels_list_tot_3_1.pkl', 'rb')   # 'rb' for reading binary file
labels_list_tot_3_1_pkl = pickle.load(f)     
f.close() 

In [0]:
f = open('labels_list_tot_4_1.pkl', 'rb')   # 'rb' for reading binary file
labels_list_tot_4_1_pkl = pickle.load(f)     
f.close() 

In [0]:
f = open('labels_list_tot_1.pkl', 'rb')   # 'rb' for reading binary file
labels_list_tot_1_pkl = pickle.load(f)     
f.close()

In [0]:
f = open('labels_list_tot_2.pkl', 'rb')   # 'rb' for reading binary file
labels_list_tot_2_pkl = pickle.load(f)     
f.close()

In [0]:
f = open('labels_list_tot_5.pkl', 'rb')   # 'rb' for reading binary file
labels_list_tot_5_pkl = pickle.load(f)     
f.close()

In [0]:
train_data_full = features25_list_1_pkl + features25_list_2_pkl + features25_list_3_1_pkl + features25_list_4_1_pkl + features25_list_5_pkl 
train_labels_full = labels_list_tot_1_pkl + labels_list_tot_2_pkl + labels_list_tot_3_1_pkl + labels_list_tot_4_1_pkl + labels_list_tot_5_pkl 


In [0]:
print(len(train_data_full), len(train_labels_full))

2409 2409


In [0]:
Batch_size = 50
Hidden_dim = 256
Num_classes = 25

train_data = train_data_full
train_labels = train_labels_full

state_dict = torch.load('feat5_4_3_s_2_s_1_s.pth')

model_1 = LSTM(Hidden_dim,Batch_size,Num_classes)
model_1.load_state_dict(state_dict)
print(model_1)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model_1.parameters(), lr=0.1)

#model_1 = model_1.to(device) 


correct = 0
train_accuracy = 0
for epoch in range(10):
    loss_count = 0.0
    correct = 0
    total = 0
    train_accuracy = 0
    start = time.time()
    #for feats, labels in train25_dataloader2_1_pkl:
    for i in range(len(train_data)//Batch_size):
      #print(feats.size())
      #i = 0
      #while i < (feats.size()[0]//25):
      #  feat_reshaped = torch.reshape(feats[25*i:25*(i+1)], (-1, 102400))
      #  labels_reshaped = torch.reshape(labels[25*i:25*(i+1)], (-1, 25))
        #print(feat_reshaped.size(), labels_reshaped.size())
        #print(feat_reshaped, labels_reshaped[0])
        #feats = input_data[i]
        #labels = label[i]
      inputs  = train_data[i: i+Batch_size]
      inputs = torch.stack(inputs)
      #inputs = inputs.to(device)
      labels = train_labels[i : i+Batch_size]
      labels = torch.stack(labels)
      #print(labels.shape)
      #labels = labels.to(device)
      model_1.zero_grad()
      model_1.hidden= model_1.init_hidden()
      outputs = model_1(inputs)
      #print(outputs.shape, labels.shape)
      loss = loss_function(outputs, labels)
      loss.backward()
      optimizer.step()
      loss_count += loss.item()
      _, predicted = torch.max(outputs.data, 1)
      
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      del outputs, inputs, labels
      #torch.cuda.empty_cache()
      train_accuracy = (correct/total)*100
      
      #i += 1 

    print('Epoch : %d Loss: %.3f' %(epoch+1, loss_count/len(train_data)))
    print('Training Accuracy', train_accuracy , time.time() - start)

LSTM(
  (lstm): LSTM(102400, 256)
  (hidden2label): Linear(in_features=256, out_features=25, bias=True)
)
Epoch : 1 Loss: 0.000
Training Accuracy 100.0 43.28739356994629
Epoch : 2 Loss: 0.000
Training Accuracy 100.0 43.76031994819641
Epoch : 3 Loss: 0.000
Training Accuracy 100.0 44.07192420959473
Epoch : 4 Loss: 0.000
Training Accuracy 100.0 44.449360609054565
Epoch : 5 Loss: 0.000
Training Accuracy 100.0 44.62438917160034
Epoch : 6 Loss: 0.000
Training Accuracy 100.0 44.9768705368042
Epoch : 7 Loss: 0.000
Training Accuracy 100.0 45.30792593955994
Epoch : 8 Loss: 0.000
Training Accuracy 100.0 45.22724461555481
Epoch : 9 Loss: 0.000
Training Accuracy 100.0 45.620566606521606
Epoch : 10 Loss: 0.000
Training Accuracy 100.0 45.61465525627136


In [0]:
train_data_full_numpy = []
train_labels_full_numpy = []

for i in range(len(test_data)):
  train_data_full_numpy.append(train_data_full[i].numpy())
  train_labels_full_numpy.append(train_labels_full[i].numpy()) 

In [0]:
#train_data_full_numpy = np.concatenate([i for i in train_data_full_numpy])
#train_labels_full_numpy = np.concatenate([i for i in train_labels_full_numpy])
train_svm = []
for i in range(len(train_data_full_numpy)):
  train_svm.append(np.concatenate(train_data_full_numpy[i])) 

In [0]:
from sklearn.metrics import accuracy_score 
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

clf =  OneVsRestClassifier(LinearSVC(random_state=None ,tol=1e-5, loss='squared_hinge', C=100, multi_class='ovr', max_iter=2000))
clf.fit(np.asarray(train_data_full_numpy), np.asarray(train_labels_full_numpy))

OneVsRestClassifier(estimator=LinearSVC(C=100, class_weight=None, dual=True,
                                        fit_intercept=True, intercept_scaling=1,
                                        loss='squared_hinge', max_iter=2000,
                                        multi_class='ovr', penalty='l2',
                                        random_state=None, tol=1e-05,
                                        verbose=0),
                    n_jobs=None)

In [0]:
pred_labels = clf.predict(np.asarray(train_data_full_numpy))
training_accuracy_svm = accuracy_score(train_labels_full_numpy, pred_labels)
print("The training accuracy is {:.2f}%".format(training_accuracy_svm*100))

The training accuracy is 100.00%


In [0]:
shuffled_test = np.asarray(test_data_numpy)
shuffled_test = np.random.shuffle(shuffled_test)
pred_labels = clf.predict(np.asarray(test_data_numpy))
testing_accuracy_svm = accuracy_test1(test_labels_numpy, pred_labels) 
print("The testing accuracy is {:.2f}%".format(testing_accuracy_svm*100))

The testing accuracy is 73.89%


In [0]:
# Don't hardcode the train and test accuracy
print('Training accuracy is %2.3f :' %(100.00), 
training_accuracy_svm*100)
print('Test accuracy is %2.3f :' %(100.00), testing_accuracy_svm*100 )

Training accuracy is 100.000 : 100.0
Test accuracy is 100.000 : 73.89473684210527


## **Problem 4.** Report

## **Bonus**


* ##### **Print the size of your training and test data**

In [0]:
df_76 = df[df.class_id > 25]

df_76_train = df_76[df_76.data_usage == 1]
df_76_test = df_76[df_76.data_usage == 2]

train76_folders = df_76_train.folder.tolist()
train76_labels = df_76_train.class_id.tolist()

test76_folders = df_76_test.folder.tolist()
test76_labels = df_76_test.class_id.tolist()


In [0]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

vgg16_local_76 = torchvision.models.vgg16(True).to(device)
del vgg16_local_76.classifier[1:]

print(vgg16_local_76.classifier)

normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
prep = torchvision.transforms.Compose([ torchvision.transforms.ToTensor(), normalize ])

In [0]:
#Code to extract Remaining 76 classes

"""
import gc
import time

training76_feats = []
training76_labels = []
root_dir_imgs = "./images"

full_time = time.time()

for idx in range(len(train76_folders)):
        start = time.time()
        folder_name = os.path.join(root_dir_imgs, train76_folders[idx])
        label = train76_labels[idx]
        
        files = os.listdir(folder_name)
        #print(files)
        #for i in range(len(files)):
        i = 0
        img125 = []
        feats25 = torch.empty(0)
        while i < 25:
          
          file_name = os.path.join(folder_name, files[i])
            
          #img_name = img_name + '/*.jpg' 
          #image = cv2.imread(file_name)
          

          #if idx % 25 == 0:
          #print(folder_name, label)
          gc.collect()
          #torch.cuda.empty_cache()

          #print(file_name)
          img = cv2.imread(file_name)
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          img = prep(img)
          #print(img.size()[0])
          img_c = img[ :, (img.shape[1]-224)//2:(img.shape[1]+224)//2, (img.shape[2]-224)//2:(img.shape[2]+224)//2]
          img_tl = img[:, :224, :224] 
          img_tr = img[:, :224, (img.shape[2]-224):]
          img_bl = img[:, (img.shape[1]-224):, :224] 
          img_br = img[:, (img.shape[1]-224):, (img.shape[2]-224):] 
          #print(len(img_tr[2]), len(img_bl[1]), len(img_br[0]))
          #print(img_c.size(), img_tl.size(), img_tr.size(), img_bl.size(), img_br.size())
          img125.extend([img_c, img_tl, img_tr, img_bl, img_br])

          i += 1

        #print("img125", len(img125))

        with torch.no_grad():
          inputs = torch.stack(img125)
          #print(len(inputs))
          inputs = inputs.to(device)
          #img_feat = self.vgg16(inputs)
          
          img_feat125 = vgg16_local_76(inputs)
          #print(img_feat125.shape)
          img_feat125 = img_feat125.cpu().detach().numpy()
          #print(inputs.is_cuda, img_feat1.is_cuda, img_c.is_cuda)#, img_feat1.is_cuda)
          del inputs
          torch.cuda.empty_cache()
          #gc.collect()
          #img_feat = self.vgg16(img_c)
          #print(len(img_feat), len(img_feat[0]),len(img_feat[1]),len(img_feat[2]))

        j = 0
        feats_video = np.zeros(0)
        while j < 25:
          feats25_mean = img_feat125[5*j:5*(j+1)]
          #print(feats25_mean.shape)
          feats25_mean = np.mean(feats25_mean, axis = 0)
          #print("feats25_mean", feats25_mean.shape)
          if j == 0:
            feats_video = feats25_mean            
          else:
            feats_video = np.concatenate((feats_video, feats25_mean), 0)
          j += 1  

        del img_feat125  
        torch.cuda.empty_cache()
        gc.collect()    

        #print("feats_video",feats_video.shape)
        training76_feats.append(feats_video)  
        training76_labels.append(label)

        if idx % 100 == 0 and idx != 0:
          print(idx)
          f = open('training76_feats_labels.pkl', 'wb')   # Pickle file is newly created where foo1.py is
          pickle.dump([training76_feats, training76_labels], f, -1)          # dump data to f
          f.close()


        print("Folders done:", len(training76_feats), label, time.time() - start, time.time() - full_time)

"""        
       

In [0]:
"""
import gc
import time
import pickle

testing76_feats = []
testing76_labels = []
root_dir_imgs = "./images"

full_time = time.time()

for idx in range(len(test76_folders)):
        start = time.time()
        folder_name = os.path.join(root_dir_imgs, test76_folders[idx])
        label = test76_labels[idx]
        
        files = os.listdir(folder_name)
        #print(files)
        #for i in range(len(files)):
        i = 0
        img125 = []
        feats25 = torch.empty(0)
        while i < 25:
          
          file_name = os.path.join(folder_name, files[i])
            
          #img_name = img_name + '/*.jpg' 
          #image = cv2.imread(file_name)
          

          #if idx % 25 == 0:
          #print(folder_name, label)
          gc.collect()
          #torch.cuda.empty_cache()

          #print(file_name)
          img = cv2.imread(file_name)
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          img = prep(img)
          #print(img.size()[0])
          img_c = img[ :, (img.shape[1]-224)//2:(img.shape[1]+224)//2, (img.shape[2]-224)//2:(img.shape[2]+224)//2]
          img_tl = img[:, :224, :224] 
          img_tr = img[:, :224, (img.shape[2]-224):]
          img_bl = img[:, (img.shape[1]-224):, :224] 
          img_br = img[:, (img.shape[1]-224):, (img.shape[2]-224):] 
          #print(len(img_tr[2]), len(img_bl[1]), len(img_br[0]))
          #print(img_c.size(), img_tl.size(), img_tr.size(), img_bl.size(), img_br.size())
          img125.extend([img_c, img_tl, img_tr, img_bl, img_br])

          i += 1

        #print("img125", len(img125))

        with torch.no_grad():
          inputs = torch.stack(img125)
          #print(len(inputs))
          inputs = inputs.to(device)
          #img_feat = self.vgg16(inputs)
          
          img_feat125 = vgg16_local_76(inputs)
          #print(img_feat125.shape)
          img_feat125 = img_feat125.cpu().detach().numpy()
          #print(inputs.is_cuda, img_feat1.is_cuda, img_c.is_cuda)#, img_feat1.is_cuda)
          del inputs
          torch.cuda.empty_cache()
          #gc.collect()
          #img_feat = self.vgg16(img_c)
          #print(len(img_feat), len(img_feat[0]),len(img_feat[1]),len(img_feat[2]))

        j = 0
        feats_video = np.zeros(0)
        while j < 25:
          feats25_mean = img_feat125[5*j:5*(j+1)]
          #print(feats25_mean.shape)
          feats25_mean = np.mean(feats25_mean, axis = 0)
          #print("feats25_mean", feats25_mean.shape)
          if j == 0:
            feats_video = feats25_mean            
          else:
            feats_video = np.concatenate((feats_video, feats25_mean), 0)
          j += 1  

        del img_feat125  
        torch.cuda.empty_cache()
        gc.collect()    

        #print("feats_video",feats_video.shape)
        testing76_feats.append(feats_video)  
        testing76_labels.append(label)

        if idx % 100 == 0 and idx != 0:
          print(idx)
          f = open('testing76_feats_labels.pkl', 'wb')   # Pickle file is newly created where foo1.py is
          pickle.dump([testing76_feats, testing76_labels], f, -1)          # dump data to f
          f.close()


        print("Folders done:", len(testing76_feats),"/", len(test76_folders), label, time.time() - start, time.time() - full_time)

"""        
       

In [0]:
# Don't hardcode the shape of train and test data
print('Shape of training data is :', )
print('Shape of test/validation data is :', )

* ##### **Modelling and evaluation**

In [0]:
#Write your code for modelling and evaluation

## Submission
---
**Runnable source code in ipynb file and a pdf report are required**.

The report should be of 3 to 4 pages describing what you have done and learned in this homework and report performance of your model. If you have tried multiple methods, please compare your results. If you are using any external code, please cite it in your report. Note that this homework is designed to help you explore and get familiar with the techniques. The final grading will be largely based on your prediction accuracy and the different methods you tried (different architectures and parameters).

Please indicate clearly in your report what model you have tried, what techniques you applied to improve the performance and report their accuracies. The report should be concise and include the highlights of your efforts.
The naming convention for report is **Surname_Givenname_SBUID_report*.pdf**

When submitting your .zip file through blackboard, please
-- name your .zip file as **Surname_Givenname_SBUID_hw*.zip**.

This zip file should include:
```
Surname_Givenname_SBUID_hw*
        |---Surname_Givenname_SBUID_hw*.ipynb
        |---Surname_Givenname_SBUID_hw*.pdf
        |---Surname_Givenname_SBUID_report*.pdf
```

For instance, student Michael Jordan should submit a zip file named "Jordan_Michael_111134567_hw5.zip" for homework5 in this structure:
```
Jordan_Michael_111134567_hw5
        |---Jordan_Michael_111134567_hw5.ipynb
        |---Jordan_Michael_111134567_hw5.pdf
        |---Jordan_Michael_111134567_report*.pdf
```

The **Surname_Givenname_SBUID_hw*.pdf** should include a **google shared link**. To generate the **google shared link**, first create a folder named **Surname_Givenname_SBUID_hw*** in your Google Drive with your Stony Brook account. 

Then right click this folder, click ***Get shareable link***, in the People textfield, enter two TA's emails: ***bo.cao.1@stonybrook.edu*** and ***sayontan.ghosh@stonybrook.edu***. Make sure that TAs who have the link **can edit**, ***not just*** **can view**, and also **uncheck** the **Notify people** box.

Colab has a good feature of version control, you should take advantage of this to save your work properly. However, the timestamp of the submission made in blackboard is the only one that we consider for grading. To be more specific, we will only grade the version of your code right before the timestamp of the submission made in blackboard. 

You are encouraged to post and answer questions on Piazza. Based on the amount of email that we have received in past years, there might be dealys in replying to personal emails. Please ask questions on Piazza and send emails only for personal issues.

Be aware that your code will undergo plagiarism check both vertically and horizontally. Please do your own work.